In [35]:
from pyspark.find_spark_home import _find_spark_home
from pyspark.sql import SparkSession
from pyspark import SparkConf, StorageLevel
from pyspark.sql.functions import col, split, expr
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType
import os

from dotenv import load_dotenv
load_dotenv()

print(_find_spark_home())

C:\Users\anama\anaconda3\envs\Bigdata\Lib\site-packages\pyspark


In [36]:
python_path = os.environ.get('PYTHON_PATH')
app_name_dec = os.environ.get('APP_NAME_DEC')
hadoop_path_dec = os.environ.get('HADOOP_DEC_DATASET_PATH')

In [37]:
conf = SparkConf()\
    .setMaster('local[*]')\
    .set('spark-local-dir', "C:\\spark-temp")\
    .set('spark.driver.memory', '4g')\
    .set('spark.executor.memory', '4g')\
    .set('spark.driver.maxResultSize', '2g')\
    .set('spark.pyspark.python', python_path)\
    .set('spark.pyspark.driver.python', python_path)\
    .set("spark.network.timeout","800s")\
    .set("spark.dynamicAllocation.enabled", "true")\
    .set("spark.shuffle.service.enabled", "true")\
    .set("spark.dynamicAllocation.minExecutors", "1")\
    .set("spark.dynamicAllocation.maxExecutors", "10")\
    .set("spark.dynamicAllocation.executorIdleTimeout", "60s")\

spark = SparkSession.builder.appName(app_name_dec).config(conf=conf).getOrCreate()
sc = spark.sparkContext

for item in sc.getConf().getAll(): print(item)

('spark.executor.extraJavaOptions', '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false')
('spark.dynamicAllocation.minExecutors', '1')
('spark.shuffle.service.enabled', 'true')
('spark.driver.memory', '4g')
('spark.executor.memory', '4g')
('spar

In [49]:
df = spark.read.csv(hadoop_path_dec, header = True, inferSchema = True)
df.show()

Py4JJavaError: An error occurred while calling o723.csv.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.lang.reflect.Constructor.newInstance(Constructor.java:423)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
py4j.ClientServerConne

In [48]:
df_index = df.select('*').withColumn('id', F.monotonically_increasing_id())
column_names = ['id'] + [col for col in df.columns]

df_index_first = df_index.select(column_names)
df_index_first.show()

RuntimeError: SparkContext or SparkSession should be created first.

In [40]:
df_new = df_index_first.drop('user_session', 'category_id')
df_new.show()

+---+-------------------+----------+----------+--------------------+-------+-------+---------+
| id|         event_time|event_type|product_id|       category_code|  brand|  price|  user_id|
+---+-------------------+----------+----------+--------------------+-------+-------+---------+
|  0|2019-12-01 01:00:00|      view|   1005105|construction.tool...|  apple|1302.48|556695836|
|  1|2019-12-01 01:00:00|      view|  22700068|                null|  force| 102.96|577702456|
|  2|2019-12-01 01:00:01|      view|   2402273|appliances.person...|  bosch| 313.52|539453785|
|  3|2019-12-01 01:00:02|  purchase|  26400248|computers.periphe...|   null| 132.31|535135317|
|  4|2019-12-01 01:00:02|      view|  20100164|    apparel.trousers|   nika| 101.68|517987650|
|  5|2019-12-01 01:00:02|      view| 100008256|accessories.umbrella|   ikea| 163.56|542860793|
|  6|2019-12-01 01:00:02|      view|  21400264|  electronics.clocks|   null|  88.81|538021416|
|  7|2019-12-01 01:00:03|      view|   1005239|con

In [41]:
df_new = df_new.withColumn("split_code", split(col("category_code"), "\."))

df_new = df_new.withColumn("category", col("split_code")[0])

df_new = df_new.withColumn("product", expr("substring(category_code, length(category) + 2)"))

df_new =df_new.drop("split_code")

df_new.show(truncate=False)

+---+-------------------+----------+----------+---------------------------------+-------+-------+---------+------------+-----------------------+
|id |event_time         |event_type|product_id|category_code                    |brand  |price  |user_id  |category    |product                |
+---+-------------------+----------+----------+---------------------------------+-------+-------+---------+------------+-----------------------+
|0  |2019-12-01 01:00:00|view      |1005105   |construction.tools.light         |apple  |1302.48|556695836|construction|tools.light            |
|1  |2019-12-01 01:00:00|view      |22700068  |null                             |force  |102.96 |577702456|null        |null                   |
|2  |2019-12-01 01:00:01|view      |2402273   |appliances.personal.massager     |bosch  |313.52 |539453785|appliances  |personal.massager      |
|3  |2019-12-01 01:00:02|purchase  |26400248  |computers.peripherals.printer    |null   |132.31 |535135317|computers   |peripheral

In [42]:
df_new =df_new.drop("category_code")
df_new.show(truncate=False)

+---+-------------------+----------+----------+-------+-------+---------+------------+-----------------------+
|id |event_time         |event_type|product_id|brand  |price  |user_id  |category    |product                |
+---+-------------------+----------+----------+-------+-------+---------+------------+-----------------------+
|0  |2019-12-01 01:00:00|view      |1005105   |apple  |1302.48|556695836|construction|tools.light            |
|1  |2019-12-01 01:00:00|view      |22700068  |force  |102.96 |577702456|null        |null                   |
|2  |2019-12-01 01:00:01|view      |2402273   |bosch  |313.52 |539453785|appliances  |personal.massager      |
|3  |2019-12-01 01:00:02|purchase  |26400248  |null   |132.31 |535135317|computers   |peripherals.printer    |
|4  |2019-12-01 01:00:02|view      |20100164  |nika   |101.68 |517987650|apparel     |trousers               |
|5  |2019-12-01 01:00:02|view      |100008256 |ikea   |163.56 |542860793|accessories |umbrella               |
|

In [43]:
df_new = df_new.withColumn("date", split(col("event_time"), " ")[0])
df_new = df_new.withColumn("day", split(col("date"), "-")[2].cast(IntegerType()))
df_new = df_new.drop("event_time")

df_new.show()

+---+----------+----------+-------+-------+---------+------------+--------------------+----------+---+
| id|event_type|product_id|  brand|  price|  user_id|    category|             product|      date|day|
+---+----------+----------+-------+-------+---------+------------+--------------------+----------+---+
|  0|      view|   1005105|  apple|1302.48|556695836|construction|         tools.light|2019-12-01|  1|
|  1|      view|  22700068|  force| 102.96|577702456|        null|                null|2019-12-01|  1|
|  2|      view|   2402273|  bosch| 313.52|539453785|  appliances|   personal.massager|2019-12-01|  1|
|  3|  purchase|  26400248|   null| 132.31|535135317|   computers| peripherals.printer|2019-12-01|  1|
|  4|      view|  20100164|   nika| 101.68|517987650|     apparel|            trousers|2019-12-01|  1|
|  5|      view| 100008256|   ikea| 163.56|542860793| accessories|            umbrella|2019-12-01|  1|
|  6|      view|  21400264|   null|  88.81|538021416| electronics|       

In [44]:
df_new = df_new.drop("date")
df_new.show()

+---+----------+----------+-------+-------+---------+------------+--------------------+---+
| id|event_type|product_id|  brand|  price|  user_id|    category|             product|day|
+---+----------+----------+-------+-------+---------+------------+--------------------+---+
|  0|      view|   1005105|  apple|1302.48|556695836|construction|         tools.light|  1|
|  1|      view|  22700068|  force| 102.96|577702456|        null|                null|  1|
|  2|      view|   2402273|  bosch| 313.52|539453785|  appliances|   personal.massager|  1|
|  3|  purchase|  26400248|   null| 132.31|535135317|   computers| peripherals.printer|  1|
|  4|      view|  20100164|   nika| 101.68|517987650|     apparel|            trousers|  1|
|  5|      view| 100008256|   ikea| 163.56|542860793| accessories|            umbrella|  1|
|  6|      view|  21400264|   null|  88.81|538021416| electronics|              clocks|  1|
|  7|      view|   1005239| xiaomi| 256.38|525740700|construction|         tools

In [45]:
df_new = df_new.dropDuplicates()

In [46]:
df_new = df_new.dropna()
df_new.show()

+------+----------+----------+------------+-----+---------+------------+-----------------+---+
|    id|event_type|product_id|       brand|price|  user_id|    category|          product|day|
+------+----------+----------+------------+-----+---------+------------+-----------------+---+
|776076|      view|   4804539|      ritmix| 0.87|512983831|  appliances|   kitchen.juicer|  1|
|473320|      view|  40500036|       inkax| 2.57|541682579|country_yard|       cultivator|  1|
|749788|      view|  58100136| organicshop| 2.65|578899590| accessories|              bag|  1|
|379554|      view|   9101106|      ritmix| 3.18|571772259|   furniture|    kitchen.table|  1|
|828981|      view|  17400135|viviennesabo| 3.19|522430846|construction|   tools.painting|  1|
|690311|      view|  24400518|    floresan|  3.2|516591584|        auto|accessories.winch|  1|
|929317|      view|  17600785|      lirene| 3.28|549085369|     apparel|            shoes|  1|
|582080|      view|  40500095|       moxom| 3.35|5

In [30]:
print("Null values present in:")
for c in ["category","product","date", "brand"]:
    print(c +':', df_new.where(F.col(c).isNull()).count())

Null values present in:
category: 0
product: 0
date: 0
brand: 0


In [31]:
count = df_new.count()
print(f"Total number of rows: {count:,d}")

Total number of rows: 53,612,307


In [32]:
df_new.describe().show()

+-------+--------------------+----------+--------------------+--------+-----------------+--------------------+-----------+-----------------+----------+------------------+
|summary|                  id|event_type|          product_id|   brand|            price|             user_id|   category|          product|      date|               day|
+-------+--------------------+----------+--------------------+--------+-----------------+--------------------+-----------+-----------------+----------+------------------+
|  count|            53612307|  53612307|            53612307|53612307|         53612307|            53612307|   53612307|         53612307|  53612307|          53612307|
|   mean|2.968796283566086...|      null|1.5950927373301918E7|     NaN|300.6398401599999|  5.48345002697422E8|       null|             null|      null|17.271609986863652|
| stddev|1.734894542916755...|      null|2.7910823423054915E7|     NaN|362.0023991418133|2.8581691692769576E7|       null|             null|     

In [47]:
save_path = hadoop_path_dec + '/selected_Data/' 

df_new.repartition(8).write.mode('overwrite').option("header", "true").csv(save_path)
spark.stop()